<a href="https://colab.research.google.com/github/Drackyma/business_data_Analitics/blob/main/%E3%83%93%E3%82%B8%E3%83%8D%E3%82%B9%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90NMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Excelファイルを読み込む
df = pd.read_excel('aa.xlsx')

# 新しい年齢階級の列を作成
df['0～9歳'] = df['0～4歳'] + df['5～9歳']
df['10～19歳'] = df['10～14歳'] + df['15～19歳']
df['20～29歳'] = df['20～24歳'] + df['25～29歳']
df['30～39歳'] = df['30～34歳'] + df['35～39歳']
df['40～49歳'] = df['40～44歳'] + df['45～49歳']
df['50～59歳'] = df['50～54歳'] + df['55～59歳']
df['60～69歳'] = df['60～64歳'] + df['65～69歳']
df['70～79歳'] = df['70～74歳'] + df['75～79歳']
df['80歳以上'] = df['80～84歳'] + df['85歳以上']

# 必要な列だけを選択
result_df = df[['全国・都道府県', '総数', '0～9歳', '10～19歳', '20～29歳',
                '30～39歳', '40～49歳', '50～59歳', '60～69歳',
                '70～79歳', '80歳以上']]

# 結果を表示
print(result_df)

# 必要に応じて結果を保存
# result_df.to_excel('結果.xlsx', index=False)

   全国・都道府県      総数  0～9歳  10～19歳  20～29歳  30～39歳  40～49歳  50～59歳  60～69歳  \
0       全国  124352  8925   10742   12715   13427   16880   17929   14839   
1      北海道    5092   318     406     453     505     674     720     678   
2      青森県    1184    73      93      87     107     152     168     177   
3      岩手県    1163    73      96      89     109     149     158     169   
4      宮城県    2264   155     197     237     247     316     308     290   
5      秋田県     914    50      68      59      77     114     120     143   
6      山形県    1026    68      87      79      98     130     133     148   
7      福島県    1767   119     149     142     178     228     238     260   
8      茨城県    2825   194     246     268     295     382     401     358   
9      栃木県    1897   130     167     175     203     262     269     246   
10     群馬県    1902   130     168     179     192     254     274     236   
11     埼玉県    7331   526     628     788     821    1027    1110     819   
12     千葉県  

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

# データの準備
data = df[['全国・都道府県', '総数', '0～4歳', '5～9歳', '10～14歳', '15～19歳',
           '20～24歳', '25～29歳', '30～34歳', '35～39歳', '40～44歳',
           '45～49歳', '50～54歳', '55～59歳', '60～64歳', '65～69歳',
           '70～74歳', '75～79歳', '80～84歳', '85歳以上']]
data = df[df['全国・都道府県'] != '全国'].copy()
# 20歳ごとの年齢階級に再集計
data['0～19歳'] = data['0～4歳'] + data['5～9歳'] + data['10～14歳'] + data['15～19歳']
data['20～39歳'] = data['20～24歳'] + data['25～29歳'] + data['30～34歳'] + data['35～39歳']
data['40～59歳'] = data['40～44歳'] + data['45～49歳'] + data['50～54歳'] + data['55～59歳']
data['60～79歳'] = data['60～64歳'] + data['65～69歳'] + data['70～74歳'] + data['75～79歳']
data['80歳以上'] = data['80～84歳'] + data['85歳以上']

# 必要な列だけを選択
data_20y = data[['全国・都道府県', '総数', '0～19歳', '20～39歳', '40～59歳', '60～79歳', '80歳以上']]

# 分析用データの準備（都道府県列を除外し、値を10で割る）
anal_data = data_20y.drop('全国・都道府県', axis=1).values / 100

# k=3から6までの分析を実行
for k in range(3, 7):
    print(f'\nk={k}のとき')
    model = NMF(n_components=k, init='random', random_state=1, solver='mu')
    model.fit(anal_data)

    W = model.fit_transform(anal_data)
    print("基底行列 W:")
    print(np.round(W, 1))

    H = model.components_
    print("\n係数行列 H:")
    print(np.round(H, 1))

    error = np.sum(abs(anal_data - np.dot(W, H)))
    print(f"\n再構成誤差: {error}")

    # k=6の場合のみ、クラスタリング結果も表示
    if k == 4:
        print("\nクラスタリング結果:")
        pref = data_20y['全国・都道府県'].values
        cluster_assignments = np.argmax(W, axis=1)
        result_df = pd.DataFrame({
            '全国・都道府県': pref,
            'クラスタ': cluster_assignments,
            '所属度': np.max(W, axis=1)
        })

        for i in range(k):
            print(f"\nクラスタ{i}の都道府県:")
            cluster_members = result_df[result_df['クラスタ'] == i].sort_values('所属度', ascending=False)
            print(cluster_members[['全国・都道府県', '所属度']].to_string())


k=3のとき
基底行列 W:
[[ 0.4  5.7  2.7]
 [ 0.8  0.6  0.3]
 [ 0.9  0.6  0.3]
 [ 1.9  0.6  1.3]
 [ 0.7  0.4  0.1]
 [ 0.8  0.5  0.3]
 [ 1.7  0.4  0.6]
 [ 2.5  0.7  1.4]
 [ 0.8  1.4  1.1]
 [ 1.6  0.6  0.9]
 [ 1.5  6.6  5.9]
 [ 1.8  5.   4.8]
 [ 3.3 10.4 16.6]
 [ 7.3  2.4  7.4]
 [ 1.   1.5  0.8]
 [ 0.5  0.7  0.4]
 [ 0.5  0.7  0.6]
 [ 0.5  0.4  0.3]
 [ 0.2  0.8  0.4]
 [ 2.   0.3  0.7]
 [ 0.6  1.7  1. ]
 [ 2.1  2.   1.8]
 [ 4.6  3.4  6. ]
 [ 1.4  0.6  0.8]
 [ 1.3  0.2  0.9]
 [ 1.8  1.   1.6]
 [ 8.   1.3  6.3]
 [ 3.6  2.5  2.9]
 [ 0.6  0.9  0.6]
 [ 0.2  0.9  0.3]
 [ 0.5  0.1  0.2]
 [ 0.2  0.6  0.2]
 [ 0.6  1.5  1.1]
 [ 1.9  1.2  1.5]
 [ 0.   1.5  0.5]
 [ 0.4  0.4  0.2]
 [ 0.9  0.2  0.3]
 [ 1.3  0.3  0.4]
 [ 0.6  0.3  0.2]
 [ 3.   2.7  3.4]
 [ 0.9  0.1  0.3]
 [ 0.2  1.4  0.4]
 [ 1.6  0.4  0.6]
 [ 0.9  0.4  0.3]
 [ 0.2  1.1  0.4]
 [ 1.6  0.3  0.4]
 [ 0.9  0.7  1. ]]

係数行列 H:
[[8.  1.4 1.1 2.  2.2 0.9]
 [7.5 1.2 0.8 1.7 2.1 0.8]
 [2.2 0.2 1.6 1.1 0.  0. ]]

再構成誤差: 48.417224836082895

k=4のとき
基底行列 W:
[[ 

/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklea

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

# データの準備（全国を除外）
data = df[df['全国・都道府県'] != '全国'].copy()
data.loc[:, '0～19歳'] = data[['0～4歳', '5～9歳', '10～14歳', '15～19歳']].sum(axis=1)
data.loc[:, '20～39歳'] = data[['20～24歳', '25～29歳', '30～34歳', '35～39歳']].sum(axis=1)
data.loc[:, '40～59歳'] = data[['40～44歳', '45～49歳', '50～54歳', '55～59歳']].sum(axis=1)
data.loc[:, '60～79歳'] = data[['60～64歳', '65～69歳', '70～74歳', '75～79歳']].sum(axis=1)
data.loc[:, '80歳以上'] = data[['80～84歳', '85歳以上']].sum(axis=1)

# 必要な列だけを選択
data_20y = data[['全国・都道府県', '総数', '0～19歳', '20～39歳', '40～59歳', '60～79歳', '80歳以上']]

# 分析用データの準備（都道府県列を除外し、値を10で割る）
anal_data = data_20y.drop('全国・都道府県', axis=1).values / 100
# k=4でNMF分析を実行
k = 5
model = NMF(n_components=k, init='random', random_state=1, solver='mu')
model.fit(anal_data)

W = model.fit_transform(anal_data)
H = model.components_

print('基底行列 W:')
print(np.round(W, 1))

print('\n係数行列 H:')
print(np.round(H, 1))

error = np.sum(abs(anal_data - np.dot(W, H)))
print(f'\n再構成誤差: {error}')

# クラスタリング結果の表示
print('\nクラスタリング結果:')
pref = data_20y['全国・都道府県'].values
cluster_assignments = np.argmax(W, axis=1)
result_df = pd.DataFrame({
    '全国・都道府県': pref,
    'クラスタ': cluster_assignments,
    '所属度': np.max(W, axis=1)
})

print('\n各クラスタの特徴（係数行列Hの列の意味）:')
print('順番: [総数, 0～19歳, 20～39歳, 40～59歳, 60～79歳, 80歳以上]')
for i in range(k):
    print(f'\nクラス{i}の特徴パターン:')
    print(np.round(H[i], 1))
    print(f'\nクラス{i}に属する都道府県:')
    cluster_members = result_df[result_df['クラスタ'] == i].sort_values('所属度', ascending=False)
    print(cluster_members[['全国・都道府県', '所属度']].to_string())

基底行列 W:
[[ 2.5  4.3  1.7  2.   2.5]
 [ 0.   1.5  0.1  0.6  0.9]
 [ 0.4  1.3  0.4  0.4  0.2]
 [ 1.4  1.1  2.6  0.2  1.3]
 [ 0.3  1.1  0.4  0.2  0.2]
 [ 0.3  1.   0.5  0.3  0.5]
 [ 0.8  1.8  0.6  0.7  0.7]
 [ 0.2  2.5  1.3  1.3  3.4]
 [ 1.6  1.4  0.2  0.9  0.4]
 [ 1.6  1.1  1.1  0.5  0.6]
 [ 4.5  4.2  2.5  3.6  5.5]
 [ 0.8  4.1  3.1  3.1  8.8]
 [16.5  2.2  4.3  7.6  7.1]
 [ 1.4  5.7  0.6  6.6 14.9]
 [ 0.5  2.2  0.4  1.   1.5]
 [ 0.5  1.   0.8  0.2  0.2]
 [ 0.9  0.7  0.6  0.3  0.3]
 [ 0.7  0.3  0.7  0.   0.2]
 [ 0.   0.7  0.3  0.4  1. ]
 [ 1.   1.8  0.3  0.9  1. ]
 [ 0.4  1.6  1.3  0.6  1.9]
 [ 2.7  2.6  2.2  0.9  0.3]
 [ 3.2  2.9  6.5  2.2  9.1]
 [ 0.5  1.1  1.4  0.4  1.8]
 [ 0.5  1.   0.7  0.6  1.3]
 [ 0.5  1.6  1.2  1.2  3.4]
 [ 5.   5.3  2.5  4.6  7.1]
 [ 0.6  4.6  0.5  3.2  6.8]
 [ 0.8  1.1  1.2  0.1  0.1]
 [ 0.2  1.   0.2  0.4  0.5]
 [ 0.4  0.4  0.2  0.2  0. ]
 [ 0.4  0.6  0.2  0.2  0.1]
 [ 1.3  1.1  1.3  0.4  0.8]
 [ 1.9  1.8  0.7  1.1  1.4]
 [ 0.3  1.4  0.2  0.6  0.9]
 [ 0.3  0.7 

/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

# データの準備
data = df[['全国・都道府県', '総数', '0～4歳', '5～9歳', '10～14歳', '15～19歳',
           '20～24歳', '25～29歳', '30～34歳', '35～39歳', '40～44歳',
           '45～49歳', '50～54歳', '55～59歳', '60～64歳', '65～69歳',
           '70～74歳', '75～79歳', '80～84歳', '85歳以上']]

# 20歳ごとの年齢階級に再集計
data.loc[:, '0～19歳'] = data[['0～4歳', '5～9歳', '10～14歳', '15～19歳']].sum(axis=1)
data.loc[:, '20～39歳'] = data[['20～24歳', '25～29歳', '30～34歳', '35～39歳']].sum(axis=1)
data.loc[:, '40～59歳'] = data[['40～44歳', '45～49歳', '50～54歳', '55～59歳']].sum(axis=1)
data.loc[:, '60～79歳'] = data[['60～64歳', '65～69歳', '70～74歳', '75～79歳']].sum(axis=1)
data.loc[:, '80歳以上'] = data[['80～84歳', '85歳以上']].sum(axis=1)

# 必要な列だけを選択
data_20y = data[['全国・都道府県', '0～19歳', '20～39歳', '40～59歳', '60～79歳', '80歳以上']]

# 分析用データの準備（都道府県列を除外し、値を10で割る）
anal_data = data_20y.drop('全国・都道府県', axis=1).values/100

# k=4のNMF分析を実行
k = 6
model = NMF(n_components=k, init='random', random_state=1, solver='mu')
model.fit(anal_data)

W = model.fit_transform(anal_data)
H = model.components_

print('基底行列 W:')
print(np.round(W, 1))

print('\n係数行列 H:')
print(np.round(H, 1))

error = np.sum(abs(anal_data - np.dot(W, H)))
print(f'\n再構成誤差: {error}')

# クラスタリング結果の表示
print('\nクラスタリング結果:')
pref = data_20y['全国・都道府県'].values
cluster_assignments = np.argmax(W, axis=1)
result_df = pd.DataFrame({
    '全国・都道府県': pref,
    'クラスタ': cluster_assignments,
    '所属度': np.max(W, axis=1)
})

# 各クラスタの特徴も表示
print('\n各クラスタの特徴（係数行列Hの列の意味）:')
print('順番: [総数, 0～19歳, 20～39歳, 40～59歳, 60～79歳, 80歳以上]')
for i in range(k):
    print(f'\nクラス{i}の特徴パターン:')
    print(np.round(H[i], 1))
    print(f'\nクラス{i}:')
    cluster_members = result_df[result_df['クラスタ'] == i].sort_values('所属度', ascending=False)
    print(cluster_members[['全国・都道府県', '所属度']].to_string())

基底行列 W:
[[39.8 19.9 33.8 45.5 39.5  0.8]
 [ 2.4  0.1  1.8  1.6  0.6  1.3]
 [ 0.6  0.1  0.2  0.3  0.3  0.2]
 [ 0.5  0.1  0.1  0.2  0.6  0. ]
 [ 0.8  0.3  0.8  0.5  0.4  0.7]
 [ 0.5  0.   0.1  0.2  0.4  0. ]
 [ 0.5  0.   0.3  0.4  0.2  0. ]
 [ 0.9  0.2  0.4  0.5  0.4  0.2]
 [ 1.1  0.5  0.6  0.5  0.7  0.7]
 [ 0.8  0.1  0.7  0.3  0.2  0.9]
 [ 0.8  0.2  0.5  0.7  0.5  0.1]
 [ 1.9  0.3  3.3  1.7  1.4  3.2]
 [ 1.6  0.9  1.9  0.9  1.9  2.3]
 [ 0.4  0.4  8.4  1.5  4.7  7.9]
 [ 1.4  3.4  0.6  1.4  5.4  0.4]
 [ 0.9  0.1  0.4  0.5  0.9  0.2]
 [ 0.5  0.   0.4  0.5  0.1  0.1]
 [ 0.4  0.1  0.4  0.4  0.2  0.2]
 [ 0.3  0.1  0.2  0.2  0.2  0.1]
 [ 0.3  0.1  0.2  0.3  0.2  0.1]
 [ 0.9  0.2  0.5  1.   0.3  0. ]
 [ 0.8  0.4  0.3  0.5  0.6  0.1]
 [ 1.3  0.5  0.7  0.9  1.3  0.3]
 [ 1.6  3.2  1.   1.8  2.6  0.3]
 [ 0.6  0.1  0.5  0.5  0.5  0.3]
 [ 0.5  0.4  0.4  0.5  0.   0.2]
 [ 0.6  0.4  0.6  0.4  1.2  0.2]
 [ 2.3  1.6  2.9  3.1  2.4  0.8]
 [ 2.   0.9  1.4  1.7  1.   1. ]
 [ 0.6  0.1  0.3  0.4  0.2  0.2]
 [

<ipython-input-28-c8a525b55d2d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, '0～19歳'] = data[['0～4歳', '5～9歳', '10～14歳', '15～19歳']].sum(axis=1)
<ipython-input-28-c8a525b55d2d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, '20～39歳'] = data[['20～24歳', '25～29歳', '30～34歳', '35～39歳']].sum(axis=1)
<ipython-input-28-c8a525b55d2d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [ ]:
pref=data["全国・都道府県"].values
pref=pd.DataFrame(pref)
belong=pd.DataFrame(np.argmax(W,axis=1))
belong=pd.concat([pref,belong],axis=1)

'クラスタリング結果の表示'
print("***k=5***")
print("クラス0：")
print(belong[belong.iloc[:, 1]==0])
print("クラス1：")
print(belong[belong.iloc[:, 1]==1])
print("クラス2：")
print(belong[belong.iloc[:, 1]==2])
print("クラス3：")
print(belong[belong.iloc[:, 1]==3])
print("クラス4：")
print(belong[belong.iloc[:, 1]==6])
print("")

***k=5***
クラス0：
Empty DataFrame
Columns: [0, 0]
Index: []
クラス1：
Empty DataFrame
Columns: [0, 0]
Index: []
クラス2：
       0  0
0     全国  2
1    北海道  2
2    青森県  2
3    岩手県  2
4    宮城県  2
5    秋田県  2
6    山形県  2
7    福島県  2
8    茨城県  2
9    栃木県  2
10   群馬県  2
11   埼玉県  2
12   千葉県  2
14  神奈川県  2
15   新潟県  2
16   富山県  2
17   石川県  2
18   福井県  2
19   山梨県  2
20   長野県  2
21   岐阜県  2
22   静岡県  2
23   愛知県  2
24   三重県  2
25   滋賀県  2
26   京都府  2
27   大阪府  2
28   兵庫県  2
29   奈良県  2
30  和歌山県  2
31   鳥取県  2
32   島根県  2
33   岡山県  2
34   広島県  2
35   山口県  2
36   徳島県  2
37   香川県  2
38   愛媛県  2
39   高知県  2
40   福岡県  2
41   佐賀県  2
42   長崎県  2
43   熊本県  2
44   大分県  2
45   宮崎県  2
46  鹿児島県  2
47   沖縄県  2
クラス3：
      0  0
13  東京都  3
クラス4：
Empty DataFrame
Columns: [0, 0]
Index: []

